# Writing a Molecular Monte Carlo Simulation

Starting today, make sure you have the functions

1. `calculate_LJ` - written in class
1. `read_xyz` - provided in class
1. `calculate_total_energy` - modified version provided in this notebook written for homework which has cutoff
1. `calculate_distance` - should be the version written for homework which accounts for periodic boundaries.
1. `calculate_tail_correction` - written for homework 


In [1]:
# add imports here
import math
import random



In [2]:
def calculate_total_energy(coordinates, box_length, cutoff):
    """
    Calculate the total energy of a set of particles using the Lennard Jones potential.
    
    Parameters
    ----------
    coordinates : list
        A nested list containing the x, y,z coordinate for each particle
    box_length : float
        The length of the box. Assumes cubic box.
    cutoff : float
        The cutoff length
    
    Returns
    -------
    total_energy : float
        The total energy of the set of coordinates.
    """
    
    total_energy = 0
    num_atoms = len(coordinates)

    for i in range(num_atoms):
        for j in range(i+1, num_atoms):
            # Calculate the distance between the particles - exercise.
            dist_ij = calculate_distance(coordinates[i], coordinates[j], box_length)

            if dist_ij < cutoff:
                # Calculate the pairwise LJ energy
                LJ_ij = calculate_LJ(dist_ij)

                # Add to total energy.
                total_energy += LJ_ij
    return total_energy

def read_xyz(filepath):
    """
    Reads coordinates from an xyz file.
    
    Parameters
    ----------
    filepath : str
       The path to the xyz file to be processed.
       
    Returns
    -------
    atomic_coordinates : list
        A two dimensional list containing atomic coordinates
    """
    
    with open(filepath) as f:
        box_length = float(f.readline().split()[0])
        num_atoms = float(f.readline())
        coordinates = f.readlines()
    
    atomic_coordinates = []
    
    for atom in coordinates:
        split_atoms = atom.split()
        
        float_coords = []
        
        # We split this way to get rid of the atom label.
        for coord in split_atoms[1:]:
            float_coords.append(float(coord))
            
        atomic_coordinates.append(float_coords)
        
    return atomic_coordinates, box_length

def calculate_LJ(r_ij):
    """
    The LJ interaction energy between two particles.

    Computes the pairwise Lennard Jones interaction energy based on the separation distance in reduced units.

    Parameters
    ----------
    r_ij : float
        The distance between the particles in reduced units.
    
    Returns
    -------
    pairwise_energy : float
        The pairwise Lennard Jones interaction energy in reduced units.

    Examples
    --------
    >>> calculate_LJ(1)
    0

    """
    
    r6_term = math.pow(1/r_ij, 6)
    r12_term = math.pow(r6_term, 2)
    
    pairwise_energy = 4 * (r12_term - r6_term)
    
    return pairwise_energy


## Add your group's modified calculate_distance function


## Add your group's tail correction function

In [3]:
def calculate_tail_correction(box_legth, n_particles, cutoff):
    """
    Calculates the tail correction
    
    Parameters
    ----------
    box_legth : float
        The distance between the particles in reduced units.
    n_particles : int
        The number of particles.
    cutoff : float
        The cutoff value.
    """
    pi_term = (8 * math.pi * math.pow(n_particles, 2)) / (3 * math.pow(box_legth, 3))
    r9_term = (1/3) * (math.pow(1/cutoff, 9))
    r3_term = math.pow(1/cutoff, 3)
    
    tail_correction = pi_term * (r9_term - r3_term)
    
    return tail_correction




In [4]:
def calculate_distance(coord1, coord2, box_length=None):
    """
    Calculate the distance between two 3D coordinates.
    
    Parameters
    ----------
    coord1, coord2: list
        The atomic coordinates
    
    Returns
    -------
    distance: float
        The distance between the two points.
    """
    
    distance = 0
    for i in range(3):
        dim_dist = (coord1[i] - coord2[i]) 
        
        if box_length:
            dim_dist = dim_dist - box_length * round(dim_dist / box_length)
        
        dim_dist = dim_dist**2
        distance += dim_dist
    
    distance = math.sqrt(distance)
    return distance

In [5]:
def accept_or_reject(delta_e, beta):
    """
    Calculates 
    
    Parameters
    ----------
    delta_e : float
        Delta value.
    beta : float
        Beta value.

    """
    if delta_e <= 0.0:
        accept = True
    else:
        random_number = random.random()
        p_acc = math.exp(-beta*delta_e)
        if random_number < p_acc:
            accept = True
        else:
            accept = False

    return accept

In [6]:
random.random()

0.292017435201229

In [7]:
delta_energy = -1
beta = 1
assert accept_or_reject(delta_energy, beta) is True

delta_energy = 0
beta = 1
assert accept_or_reject(delta_energy, beta) is True

In [8]:
random.seed(5)
random.random()

0.6229016948897019

In [9]:
random.seed(0)
random.random()

0.8444218515250481

In [10]:
random.seed(0)
delta_e = 1
beta = 1
assert accept_or_reject(delta_e, beta) is False

In [11]:
random.seed(1)
delta_e = 1
beta = 1
assert accept_or_reject(delta_e, beta) is True

In [12]:
random.seed()

In [13]:
def calculate_pair_energy(coordinates, i_particle, box_length, cutoff):
    """
    Calculate the interaction energy of a particle with its environment (all other particles in the system)
    
    Parameters
    ----------
    coordinates : list
        The coordinates for all particles in the system
        
    i_particle : int
        The particle number for which to calculate the energy
        
    cutoff : float
        The simulation cutoff. Beyond this distance, interactions are not calculated.
    
    Returns
    -------
    e_total : float 
        The pairwise interaction energy of he i_th particle with all other particles in the system.
    """
    
    e_total = 0.0
    i_position = coordinates[i_particle]
 
    num_atoms = len(coordinates)

    for j_particle in range(num_atoms):
        if i_particle != j_particle:

            j_position = coordinates[j_particle]
            rij = calculate_distance(i_position, j_position, box_length)

            if rij < cutoff:
                e_pair = calculate_LJ(rij)
                e_total += e_pair

    return e_total

In [14]:
## Sanity checks
test_coords = [[0,0,0], [0, 0, 2**(1/6)], [0, 0, 2*2**(1/6)]]

# What do you expect the result to be for particle index 1 (use cutoff of 3)
assert calculate_pair_energy(test_coords, 1, 10, 3) == -2

# What do you expect the result to be for particle index 0 with a cut off of 2?
assert calculate_pair_energy(test_coords, 0, 10, 2) == -1

assert calculate_pair_energy(test_coords, 0, 10, 3) == calculate_pair_energy(test_coords, 2, 10, 3)

In [15]:
def read_xyz(filepath):
    """
    Reads coordinates from an xyz file.
    
    Parameters
    ----------
    filepath : str
       The path to the xyz file to be processed.
       
    Returns
    -------
    atomic_coordinates : list
        A two dimensional list containing atomic coordinates
    """
    
    with open(filepath) as f:
        box_length = float(f.readline().split()[0])
        num_atoms = float(f.readline())
        coordinates = f.readlines()
    
    atomic_coordinates = []
    
    for atom in coordinates:
        split_atoms = atom.split()
        
        float_coords = []
        
        # We split this way to get rid of the atom label.
        for coord in split_atoms[1:]:
            float_coords.append(float(coord))
            
        atomic_coordinates.append(float_coords)
        
    
    return atomic_coordinates, box_length


In [16]:
coordinates = [[0, 0, 0], [0, 0, 2**(1/6)], [0, 0, 2*(2**(1/6))]]

assert calculate_pair_energy(coordinates, 0, 0, 1) == 0

assert calculate_pair_energy(coordinates, 1, 10, 3) == -2

assert calculate_pair_energy(coordinates, 0, 10, 3) == calculate_pair_energy(coordinates, 2, 10, 3)

assert calculate_pair_energy(coordinates, 0, 10, 2) == -1

# Monte Carlo Loop

In [17]:
# Read coordinates
coordinates, box_length = read_xyz("lj_sample_configurations/lj_sample_config_periodic1.txt")

# Set simulation parameters

reduced_temperature = 0.9
num_steps = 5000
max_displacement = 0.1
cutoff = 3.0
freq = 1000

# Calculated quantities
beta = 1 / reduced_temperature
num_particles = len(coordinates)

In [18]:
print(box_length)
print(num_particles)

# Enerrgy calculations
total_energy = calculate_total_energy(coordinates, box_length, cutoff)
print(total_energy)
c = len(coordinates)

# With tail correction
tail_correction = calculate_tail_correction(box_length, num_particles, cutoff)
print(tail_correction)
total_energy += tail_correction
print(total_energy)
                                          

10.0
800
-4351.540194543858
-198.4888837441566
-4550.029078288015


In [21]:
steps = []
energies = []

random.seed(0)
for step in range(num_steps):
    
    # 1. Randomly pick one of N particles
    random_particle = random.randrange(num_particles)

    # 2. Calculate the interaction energy of the selected particle with the system and store this value.
    current_energy = calculate_pair_energy(coordinates, random_particle, box_length, cutoff)
    
    # 3. Generate random x, y, z displacement
    x_rand = random.uniform(-max_displacement, max_displacement)
    y_rand = random.uniform(-max_displacement, max_displacement)
    z_rand = random.uniform(-max_displacement, max_displacement)
    
    # 4. Modify coordinate of Nth particle by generated displacements
    coordinates[random_particle][0] += x_rand
    coordinates[random_particle][1] += y_rand
    coordinates[random_particle][2] += z_rand
     
        
    # 5. Calculate interaction energy of moved particle with the system and store this value.
    proposed_energy = calculate_pair_energy(coordinates, random_particle, box_length, cutoff)
    #print("proposed_energy: ", proposed_energy)
    #print("current_energy:", current_energy)
    delta_energy = proposed_energy - current_energy
    
    # 6. Calculate if we accept the move
    accept = accept_or_reject(delta_energy, beta)
    
    # 7. if accept, move particle
    if accept:
        total_energy += delta_energy
        
    else:
        # Move is not accepted, roll back coordinates
        coordinates[random_particle][0] -= x_rand
        coordinates[random_particle][1] -= y_rand
        coordinates[random_particle][2] -= z_rand
    
    # 8. print energy if step is multiple of freq
    if step % freq == 0:
        print(step, total_energy/num_particles)
        # steps.append(step)
        # energies.append(total_energy/num_particles)
    
    

0 -5.696977853518065
1000 -5.701470201950446
2000 -5.651504811416864
3000 -5.653911031570111
4000 -5.663677730086829


In [20]:
len(coordinates)

800